In [1]:
# 필요한 라이브러리 가져오기

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import missingno as msno
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
# 파일 받기, Id -> Index로 사용

df_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv', index_col='Id')
df_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv', index_col='Id')
submission = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv', index_col='Id')

In [3]:
submission.head() # SalePrice

,SalePrice
Id,
1461,169277.052498
1462,187758.393989
1463,183583.683570
1464,179317.477511
1465,150730.079977


In [4]:
df_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 1461 to 2919
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1459 non-null   int64  
 1   MSZoning       1455 non-null   object 
 2   LotFrontage    1232 non-null   float64
 3   LotArea        1459 non-null   int64  
 4   Street         1459 non-null   object 
 5   Alley          107 non-null    object 
 6   LotShape       1459 non-null   object 
 7   LandContour    1459 non-null   object 
 8   Utilities      1457 non-null   object 
 9   LotConfig      1459 non-null   object 
 10  LandSlope      1459 non-null   object 
 11  Neighborhood   1459 non-null   object 
 12  Condition1     1459 non-null   object 
 13  Condition2     1459 non-null   object 
 14  BldgType       1459 non-null   object 
 15  HouseStyle     1459 non-null   object 
 16  OverallQual    1459 non-null   int64  
 17  OverallCond    1459 non-null   int64  
 18  YearB

>* feature가 총 79개

In [6]:
df_train.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1459.000000,1232.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1444.000000,1458.000000,1458.000000,...,1458.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,57.378341,68.580357,9819.161069,6.078821,5.553804,1971.357779,1983.662783,100.709141,439.203704,52.619342,...,472.768861,93.174777,48.313914,24.243317,1.794380,17.064428,1.744345,58.167923,6.104181,2007.769705
std,42.746880,22.376841,4955.517327,1.436812,1.113740,30.390071,21.130467,177.625900,455.268042,176.753926,...,217.048611,127.744882,68.883364,67.227765,20.207842,56.609763,30.491646,630.806978,2.722432,1.301740
min,20.000000,21.000000,1470.000000,1.000000,1.000000,1879.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,20.000000,58.000000,7391.000000,5.000000,5.000000,1953.000000,1963.000000,0.000000,0.000000,0.000000,...,318.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000
50%,50.000000,67.000000,9399.000000,6.000000,5.000000,1973.000000,1992.000000,0.000000,350.500000,0.000000,...,480.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,70.000000,80.000000,11517.500000,7.000000,6.000000,2001.000000,2004.000000,164.000000,753.500000,0.000000,...,576.000000,168.000000,72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,190.000000,200.000000,56600.000000,10.000000,9.000000,2010.000000,2010.000000,1290.000000,4010.000000,1526.000000,...,1488.000000,1424.000000,742.000000,1012.000000,360.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000


## 1.1 Null Data Check

In [7]:
for col in df_train.columns:
    s = 'column {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * df_train[col].isnull().sum() / df_train[col].shape[0])
    print(s)

column MSSubClass	 Percent of NaN value: 0.00%
column   MSZoning	 Percent of NaN value: 0.27%
column LotFrontage	 Percent of NaN value: 15.56%
column    LotArea	 Percent of NaN value: 0.00%
column     Street	 Percent of NaN value: 0.00%
column      Alley	 Percent of NaN value: 92.67%
column   LotShape	 Percent of NaN value: 0.00%
column LandContour	 Percent of NaN value: 0.00%
column  Utilities	 Percent of NaN value: 0.14%
column  LotConfig	 Percent of NaN value: 0.00%
column  LandSlope	 Percent of NaN value: 0.00%
column Neighborhood	 Percent of NaN value: 0.00%
column Condition1	 Percent of NaN value: 0.00%
column Condition2	 Percent of NaN value: 0.00%
column   BldgType	 Percent of NaN value: 0.00%
column HouseStyle	 Percent of NaN value: 0.00%
column OverallQual	 Percent of NaN value: 0.00%
column OverallCond	 Percent of NaN value: 0.00%
column  YearBuilt	 Percent of NaN value: 0.00%
column YearRemodAdd	 Percent of NaN value: 0.00%
column  RoofStyle	 Percent of NaN value: 0.00%
col

In [8]:
for col in df_test.columns:
    s = 'column {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * df_test[col].isnull().sum() / df_test[col].shape[0])
    print(s)

column MSSubClass	 Percent of NaN value: 0.00%
column   MSZoning	 Percent of NaN value: 0.27%
column LotFrontage	 Percent of NaN value: 15.56%
column    LotArea	 Percent of NaN value: 0.00%
column     Street	 Percent of NaN value: 0.00%
column      Alley	 Percent of NaN value: 92.67%
column   LotShape	 Percent of NaN value: 0.00%
column LandContour	 Percent of NaN value: 0.00%
column  Utilities	 Percent of NaN value: 0.14%
column  LotConfig	 Percent of NaN value: 0.00%
column  LandSlope	 Percent of NaN value: 0.00%
column Neighborhood	 Percent of NaN value: 0.00%
column Condition1	 Percent of NaN value: 0.00%
column Condition2	 Percent of NaN value: 0.00%
column   BldgType	 Percent of NaN value: 0.00%
column HouseStyle	 Percent of NaN value: 0.00%
column OverallQual	 Percent of NaN value: 0.00%
column OverallCond	 Percent of NaN value: 0.00%
column  YearBuilt	 Percent of NaN value: 0.00%
column YearRemodAdd	 Percent of NaN value: 0.00%
column  RoofStyle	 Percent of NaN value: 0.00%
col

In [9]:
# NaN이 70%가 넘는 column 확인 (train)
# 리스트 만들어서 해당 col 저장

over70NaN = []
for col in df_train.columns:
    percentOfNaN = df_train[col].isnull().sum() / df_train[col].shape[0] * 100
    if(percentOfNaN > 70):
        print('Percent of NaN', '{:<13}'.format(col), ':', percentOfNaN)
        over70NaN.append(col)

Percent of NaN Alley         : 92.66620973269363
Percent of NaN PoolQC        : 99.7943797121316
Percent of NaN Fence         : 80.12337217272105
Percent of NaN MiscFeature   : 96.50445510623715


In [10]:
# NaN이 70%가 넘는 column 확인 (test)
for col in df_train.columns:
    percentOfNaN = df_test[col].isnull().sum() / df_test[col].shape[0] * 100
    if(percentOfNaN > 70):
        print('Percent of NaN', '{:<13}'.format(col), ':', percentOfNaN)

Percent of NaN Alley         : 92.66620973269363
Percent of NaN PoolQC        : 99.7943797121316
Percent of NaN Fence         : 80.12337217272105
Percent of NaN MiscFeature   : 96.50445510623715


In [11]:
over70NaN

['Alley', 'PoolQC', 'Fence', 'MiscFeature']

>* NaN이 70%가 넘는 column은 train과 test 둘 다 ['Alley', 'PoolQC', 'Fence', 'MiscFeature']로 겹친다.

## 1.2 Target label 확인

In [12]:
# data를 한 번에 쓰도록 merge를 통해 df_train에 병합

df_train = pd.merge(df_train, submission, on='Id', how='left')